# 第8章: ニューラルネット

## 70. 単語埋め込みの読み込み

In [1]:
# 学習済み単語ベクトルのダウンロード
!pip install gdown
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM -O data/GoogleNews-vectors-negative300.bin.gz

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: lightning-lite 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of lightning-lite or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
/home/j329nish/.local/lib/python3.10/

In [2]:
!pip install numpy==1.24.3 gensim

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 98.2 MB/s eta 0:00:00:00:0100:01
DEPRECATION: lightning-lite 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of lightning-lite or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: numpy
    Found existing installation: numpy 1.2

In [3]:
import numpy as np
from gensim.models import KeyedVectors

# モデルを読み込む
model_path = "./data/GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

vocab_size = len(model.key_to_index) + 1
embedding_dim = model.vector_size

embedding_matrix = np.zeros((vocab_size, embedding_dim))

token2id = {"<PAD>": 0}
id2token = {0: "<PAD>"}

for idx, word in enumerate(model.key_to_index, start=1):
  embedding_matrix[idx] = model[word]
  token2id[word] = idx
  id2token[idx] = word

print(f"Embedding matrix shape: {embedding_matrix.shape}")

# 属性一覧の表示  https://aiacademy.jp/media/?p=2013
# 単語埋め込み    https://nlp100.vercel.app/docs/ch08/ch08.html

Embedding matrix shape: (3000001, 300)


## 71. データセットの読み込み

In [4]:
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip -P data/
!unzip -o data/SST-2.zip -d data/
!rm data/SST-2.zip

--2025-04-24 00:37:57--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
dl.fbaipublicfiles.com (dl.fbaipublicfiles.com) をDNSに問いあわせています... 13.33.5.129, 13.33.5.44, 13.33.5.53, ...
dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.33.5.129|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 7439277 (7.1M) [application/zip]
`data/SST-2.zip' に保存中

SST-2.zip           100%[===================>]   7.09M  --.-KB/s    in 0.09s   

2025-04-24 00:37:57 (78.8 MB/s) - `data/SST-2.zip' へ保存完了 [7439277/7439277]

Archive:  data/SST-2.zip
  inflating: data/SST-2/dev.tsv      
  inflating: data/SST-2/original/README.txt  
  inflating: data/SST-2/original/SOStr.txt  
  inflating: data/SST-2/original/STree.txt  
  inflating: data/SST-2/original/datasetSentences.txt  
  inflating: data/SST-2/original/datasetSplit.txt  
  inflating: data/SST-2/original/dictionary.txt  
  inflating: data/SST-2/original/original_rt_snippets.txt  
  inflating: data/SST-2/original/sentiment_labels.txt  
  

In [5]:
import csv
import collections
import torch

def make_dict(file_name):
  dictionary = []
  with open(file_name, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader)
    for row in reader:
      input_ids = [token2id[word] for word in row[0].split() if word in token2id]
      if len(input_ids) == 0:
        continue
      dictionary.append({
        'text': row[0], 
        'label': torch.tensor([float(row[1])]), 
        'input_ids': torch.tensor(input_ids, dtype=torch.long)
      })
  return dictionary

train_dict =  make_dict('./data/SST-2/train.tsv')
dev_dict = make_dict('./data/SST-2/dev.tsv')

print('学習データのリスト：')
for i in range(5):
  print(train_dict[i])
print('')

print('検証データのリスト：')
for i in range(5):
  print(dev_dict[i])

# torchのtensorについて https://qiita.com/mathlive/items/241bfb42d852bb801b96

学習データのリスト：
{'text': 'hide new secretions from the parental units ', 'label': tensor([0.]), 'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])}
{'text': 'contains no wit , only labored gags ', 'label': tensor([0.]), 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
{'text': 'that loves its characters and communicates something rather beautiful about human nature ', 'label': tensor([1.]), 'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,
         1964])}
{'text': 'remains utterly satisfied to remain the same throughout ', 'label': tensor([0.]), 'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}
{'text': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'label': tensor([0.]), 'input_ids': tensor([    6,    12,  1445, 43789,    12, 10946,    76, 41349,    42])}

検証データのリスト：
{'text': "it 's a charming and often affecting journey . ", 'label': tensor([1.]), 'input

## 72. Bag of wordsモデルの構築

In [6]:
import torch.nn as nn

class LogisticRegression(nn.Module):
  def __init__(self, embedding_matrix, freeze_embedding):
    super().__init__()
    self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=freeze_embedding)
    self.linear = nn.Linear(embedding_matrix.shape[1], 1)

  def forward(self, x):
    x = self.embedding(x)
    x = torch.mean(x, dim=0)
    y = self.linear(x)
    return y

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LogisticRegression(embedding_matrix, True).to(device)

# ロジスティック回帰モデルの構築 https://iifx.dev/ja/articles/315600010
# torchによるモデルの構築       https://qiita.com/h-hosoda-ml/items/28433b921ed9d3840b03
# embeddingのよる単語埋め込み   https://gotutiyan.hatenablog.com/entry/2020/09/02/200144
# torchを使った平均ベクトル     https://runebook.dev/ja/articles/pytorch/generated/torch.mean

## 73. モデルの学習

In [7]:
num_epochs = 5
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

for epoch in range(num_epochs):
  model.train()
  train_loss = []
  for train_data in train_dict:
    x = train_data['input_ids'].to(device)
    y = train_data['label'].to(device)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  model.eval()
  dev_loss = []
  with torch.no_grad():
    for dev_data in dev_dict:
      x = dev_data['input_ids'].to(device)
      y = dev_data['label'].to(device)
      y_pred = model(x)
      loss = criterion(y_pred, y)
      dev_loss.append(loss.item())
  
  print(f'[epoch {epoch + 1}/{num_epochs}] train loss: {np.mean(train_loss):.4f}, dev loss: {np.mean(dev_loss):.4f}')

torch.save(model, "./model/model_73.pt")

# モデルの学習 https://iifx.dev/ja/articles/277414431
# モデルの保存 https://pystyle.info/pytorch-save-and-load-model/

[epoch 1/5] train loss: 0.4083, dev loss: 0.4717
[epoch 2/5] train loss: 0.3815, dev loss: 0.4689
[epoch 3/5] train loss: 0.3802, dev loss: 0.4685
[epoch 4/5] train loss: 0.3799, dev loss: 0.4684
[epoch 5/5] train loss: 0.3798, dev loss: 0.4684


## 74. モデルの評価

In [8]:
from sklearn.metrics import accuracy_score

model = torch.load("./model/model_73.pt").to(device)
model.eval()
y_pred_list, y_list = [], []
for dev_data in dev_dict:
  x = dev_data['input_ids'].to(device)
  y = float(dev_data['label'].to(device))
  y_pred = float(torch.sigmoid(model(x)) > 0.5)
  y_list.append(y)
  y_pred_list.append(y_pred)

print(accuracy_score(y_list, y_pred_list))

0.7935779816513762


## 75. パディング

In [9]:
def collate(data_dict):
  data_dict.sort(key=lambda x: len(x['input_ids']), reverse=True)
  input_ids_list = [data['input_ids'] for data in data_dict]
  label_list = [data['label'] for data in data_dict]
  padded_input_ids = nn.utils.rnn.pad_sequence(input_ids_list, batch_first=True, padding_value=0)
  padded_data = {
    'input_ids': padded_input_ids,
    'label': torch.stack(label_list)
  }
  return padded_data

padded_data = collate(train_dict[0:4])
print(padded_data)

# ラベルを基準にソート https://www.headboost.jp/python-how-to-sort-a-list/
# パディング          https://qiita.com/Y_kazuyan/items/aec5519a81cfd3e98088

{'input_ids': tensor([[     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,
           1276,   1964],
        [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,
              0,      0],
        [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,
              0,      0],
        [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,
              0,      0]]), 'label': tensor([[1.],
        [0.],
        [0.],
        [0.]])}


## 76. ミニバッチ学習

In [10]:
from torch.utils.data import DataLoader

# ハイパーパラメータ
batch_size = 32
num_epochs = 5

# データの準備
train_loader = DataLoader(train_dict, batch_size=batch_size, shuffle=True, collate_fn=collate)
dev_loader = DataLoader(dev_dict, batch_size=batch_size, shuffle=False, collate_fn=collate)

# モデルの定義
class LogisticRegression(nn.Module):
  def __init__(self, embedding_matrix, freeze_embedding):
    super().__init__()
    self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=freeze_embedding)
    self.linear = nn.Linear(embedding_matrix.shape[1], 1)

  def forward(self, x):
    x = self.embedding(x)
    x = torch.mean(x, dim=1)
    y = self.linear(x)
    return y

model = LogisticRegression(embedding_matrix, True).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

# 学習
for epoch in range(num_epochs):
  model.train()
  train_loss = []
  for batch in train_loader:
    x = batch['input_ids'].to(device)
    y = batch['label'].to(device)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  model.eval()
  dev_loss, y_pred_list, y_list = [], [], []
  with torch.no_grad():
    for batch in dev_loader:
      x = batch['input_ids'].to(device)
      y = batch['label'].to(device)
      y_pred = model(x)
      loss = criterion(y_pred, y)
      dev_loss.append(loss.item())
      y_list.extend(y.cpu().numpy())
      y_pred_list.extend((y_pred > 0.5).float().cpu().numpy())
    
  print(f'[epoch {epoch + 1}/{num_epochs}] train loss: {np.mean(train_loss):.4f}, dev loss: {np.mean(dev_loss):.4f}, acc: {accuracy_score(y_list, y_pred_list):.4f}')

torch.save(model, "./model/model_76.pt")

# ミニバッチ学習 https://tma15.github.io/blog/2020/03/10/pytorch%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86-%E3%82%88%E3%82%8A%E5%B0%91%E3%81%AA%E3%81%84%E3%83%91%E3%83%87%E3%82%A3%E3%83%B3%E3%82%B0%E3%81%A7%E3%83%9F%E3%83%8B%E3%83%90%E3%83%83%E3%83%81%E5%AD%A6%E7%BF%92%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95/
# 複数append    https://af-e.net/python-append-multiple/

[epoch 1/5] train loss: 0.6197, dev loss: 0.5774, acc: 0.7179
[epoch 2/5] train loss: 0.5384, dev loss: 0.5272, acc: 0.7569
[epoch 3/5] train loss: 0.4963, dev loss: 0.5023, acc: 0.7741
[epoch 4/5] train loss: 0.4708, dev loss: 0.4944, acc: 0.7970
[epoch 5/5] train loss: 0.4553, dev loss: 0.4858, acc: 0.8005


## 77. GPU上での学習

In [11]:
model = LogisticRegression(embedding_matrix, True).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

for epoch in range(num_epochs):
  model.train()
  train_loss = []
  for batch in train_loader:
    x = batch['input_ids'].to(device)
    y = batch['label'].to(device)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  model.eval()
  dev_loss, y_pred_list, y_list = [], [], []
  with torch.no_grad():
    for batch in dev_loader:
      x = batch['input_ids'].to(device)
      y = batch['label'].to(device)
      y_pred = model(x)
      loss = criterion(y_pred, y)
      dev_loss.append(loss.item())
      y_list.extend(y.cpu().numpy())
      y_pred_list.extend((y_pred > 0.5).float().cpu().numpy())
    
  print(f'[epoch {epoch + 1}/{num_epochs}] train loss: {np.mean(train_loss):.4f}, dev loss: {np.mean(dev_loss):.4f}, acc: {accuracy_score(y_list, y_pred_list):.4f}')

torch.save(model, "./model/model_77.pt")

[epoch 1/5] train loss: 0.6181, dev loss: 0.5748, acc: 0.7133
[epoch 2/5] train loss: 0.5375, dev loss: 0.5261, acc: 0.7534
[epoch 3/5] train loss: 0.4960, dev loss: 0.5035, acc: 0.7798
[epoch 4/5] train loss: 0.4712, dev loss: 0.4909, acc: 0.7970
[epoch 5/5] train loss: 0.4546, dev loss: 0.4823, acc: 0.7993


## 78. 単語埋め込みのファインチューニング

In [12]:
model = LogisticRegression(embedding_matrix, False).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

for epoch in range(num_epochs):
  model.train()
  train_loss = []
  for batch in train_loader:
    x = batch['input_ids'].to(device)
    y = batch['label'].to(device)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  model.eval()
  dev_loss, y_pred_list, y_list = [], [], []
  with torch.no_grad():
    for batch in dev_loader:
      x = batch['input_ids'].to(device)
      y = batch['label'].to(device)
      y_pred = model(x)
      loss = criterion(y_pred, y)
      dev_loss.append(loss.item())
      y_list.extend(y.cpu().numpy())
      y_pred_list.extend((y_pred > 0.5).float().cpu().numpy())
    
  print(f'[epoch {epoch + 1}/{num_epochs}] train loss: {np.mean(train_loss):.4f}, dev loss: {np.mean(dev_loss):.4f}, acc: {accuracy_score(y_list, y_pred_list):.4f}')

torch.save(model, "./model/model_78.pt")

[epoch 1/5] train loss: 0.4195, dev loss: 0.4373, acc: 0.8142
[epoch 2/5] train loss: 0.2457, dev loss: 0.5252, acc: 0.8119
[epoch 3/5] train loss: 0.2076, dev loss: 0.5939, acc: 0.8188
[epoch 4/5] train loss: 0.1889, dev loss: 0.6476, acc: 0.8131
[epoch 5/5] train loss: 0.1790, dev loss: 0.7154, acc: 0.8096


## 79. アーキテクチャの変更

In [13]:
# ハイパーパラメータ
batch_size = 32
hidden_size = 50
num_epochs = 5

# データの準備
train_loader = DataLoader(train_dict, batch_size=batch_size, shuffle=True, collate_fn=collate)
dev_loader = DataLoader(dev_dict, batch_size=batch_size, shuffle=False, collate_fn=collate)

# モデルの定義
class LSTM(nn.Module):
  def __init__(self, embedding_matrix, freeze_embedding):
    super().__init__()
    self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=freeze_embedding)
    self.num_layers = 2
    self.lstm = nn.LSTM(
      embedding_matrix.shape[1],
      hidden_size, 
      num_layers=self.num_layers,
      batch_first=True,
      bidirectional=True
    )
    self.linear = nn.Linear(hidden_size * 2, 1)

  def forward(self, x):
    x = self.embedding(x)
    x, (h,c) = self.lstm(x)
    x = torch.cat([h[-2], h[-1]], dim=1)
    y = self.linear(x)
    return y

model = LSTM(embedding_matrix, False).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

# 学習
for epoch in range(num_epochs):
  model.train()
  train_loss = []
  for batch in train_loader:
    x = batch['input_ids'].to(device)
    y = batch['label'].to(device)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  model.eval()
  dev_loss, y_pred_list, y_list = [], [], []
  with torch.no_grad():
    for batch in dev_loader:
      x = batch['input_ids'].to(device)
      y = batch['label'].to(device)
      y_pred = model(x)
      loss = criterion(y_pred, y)
      dev_loss.append(loss.item())
      y_list.extend(y.cpu().numpy())
      y_pred_list.extend((y_pred > 0.5).float().cpu().numpy())
    
  print(f'[epoch {epoch + 1}/{num_epochs}] train loss: {np.mean(train_loss):.4f}, dev loss: {np.mean(dev_loss):.4f}, acc: {accuracy_score(y_list, y_pred_list):.4f}')

torch.save(model, "./model/model_79.pt")

# LSTMの作り方1 https://qiita.com/norimaki-senbei/items/186638b558a6dd425921
# LSTMの作り方2 https://tma15.github.io/blog/2020/03/15/%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86-lstm%E3%81%AB%E5%9F%BA%E3%81%A5%E3%81%8F%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E5%AD%A6%E7%BF%92-pytorch%E3%82%B3%E3%83%BC%E3%83%89%E4%BB%98%E3%81%8D/
# LSTMの作り方3 https://iifx.dev/ja/articles/260750715
# 双方向LSTM    https://qiita.com/m__k/items/78a5125d719951ca98d3

[epoch 1/5] train loss: 0.2740, dev loss: 0.4266, acc: 0.8119
[epoch 2/5] train loss: 0.1472, dev loss: 0.5062, acc: 0.8360
[epoch 3/5] train loss: 0.1010, dev loss: 0.5242, acc: 0.8257
[epoch 4/5] train loss: 0.0770, dev loss: 0.5664, acc: 0.8119
[epoch 5/5] train loss: 0.0600, dev loss: 0.7231, acc: 0.8050
